In [1]:
import numpy as np
import pandas as pd
import os
import tqdm
import pickle
from tqdm import tqdm

from functools import lru_cache

from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

from labeling_functions import get_all_lfs

import glob

2023-11-06 16:38:14.106689: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-06 16:38:14.142132: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-06 16:38:14.142927: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-06 16:38:14.870979: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


resources get initialised


Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [2]:
with open("workspace/datasets/final_combined_with_index.pkl", 'rb') as f:
    dataset = pickle.load(f)

In [3]:
ABSTAIN = -1
SIMPLE = 0
NOT_SIMPLE = 1
LOST_MEANING = 2

label_map = {-1: "ABSTAIN", 0: "SIMPLE", 1: "NOT_SIMPLE", 2: "LOST_MEANING"}

In [4]:
dataset['simplified_snt'] = dataset['simp']
dataset['source_snt'] = dataset['src']

In [5]:
all_lfs = get_all_lfs()

In [6]:
label_path = "/workspace/datasets/3mlabels/"

In [7]:
if not os.path.isdir(label_path):
    os.mkdir(label_path)

In [8]:
def get_finished_batches(path = "/workspace/datasets/3mlabels/*"):
    label_paths = glob.glob(path)
    fin_batches = [int(path.split("_")[-1]) for path in label_paths]
    return set(fin_batches)

In [9]:
get_finished_batches()

set()

In [14]:
batch_size = 100
start = 0

with tqdm(total=len(dataset)) as pbar:
    for i in range(start, len(dataset), batch_size):

        if not i in get_finished_batches():
            try:
                print(f"working on batch {i}")
                applier = PandasLFApplier(all_lfs)
                labels = applier.apply(dataset[i:i+batch_size])

                pickle.dump(labels, open(f"/workspace/datasets/3mlabels/labels_{i}", "wb"))

            except:
                print(f"something went wrong with batch {i}")

        pbar.update(batch_size)

  0%|          | 0/3006971 [00:00<?, ?it/s]

working on batch 20


  0%|          | 40/3006971 [00:24<520:26:56,  1.60it/s]

working on batch 40


  0%|          | 60/3006971 [00:50<752:54:02,  1.11it/s]

...

working on batch 80


  0%|          | 100/3006971 [01:41<934:31:49,  1.12s/it]

working on batch 100


  0%|          | 120/3006971 [02:00<875:47:36,  1.05s/it]

something went wrong with batch 100
working on batch 120
